In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import conda_installer
from rdkit import Chem
from deepchem.feat.graph_features import atom_features as get_atom_features
import rdkit
import pickle

2025-08-11 19:08:38.221425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 19:08:38.300304: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normaliz

In [2]:
PDBs = {}
df_info = pd.read_csv("/home/lthoma21/BFE-Loss-Function/FINAL-PDBBIND-FILES/Datasets/pdbbind.csv")

In [3]:
len(df_info)

2914

In [4]:
df_info.dropna(how='any', inplace=True)

In [5]:
len(df_info)

2735

In [ ]:
new=df_info[:300]
new.tail()

In [6]:
df_info.columns

Index(['complex-name', 'gb-complex-etot', 'gb-complex-1-4-eel',
       'gb-complex-eelec', 'gb-complex-egb', 'gb-complex-esurf',
       'gb-protein-etot', 'gb-protein-1-4-eel', 'gb-protein-eelect',
       'gb-protein-egb', 'gb-protein-esurf', 'gb-ligand-etot',
       'gb-ligand-1-4-eel', 'gb-ligand-eelec', 'gb-ligand-egb',
       'gb-ligand-esurf', 'pb-complex-etot', 'pb-complex-vdwaals',
       'pb-complex-eelect', 'pb-complex-epb', 'pb-complex-ecavity',
       'pb-protein-etot', 'pb-protein-vdwaals', 'pb-protein-eelec',
       'pb-protein-epb', 'pb-protein-ecavity', 'pb-ligand-etot',
       'pb-ligand-vdwaals', 'pb-ligand-eelec', 'pb-ligand-epb',
       'pb-ligand-ecavity', 'ddg', 'enthalpy-gb', 'entropy'],
      dtype='object')

In [7]:
#columns of physics based parameters 
[c for c in df_info.columns if ('etot' not in c) and ('epb' not in c) and ('ecavity' not in c)and ('enthalpy-gb' not in c) and ('entropy' not in c) and ('gb-' in c or 'vdwaals' in c)]


['gb-complex-1-4-eel',
 'gb-complex-eelec',
 'gb-complex-egb',
 'gb-complex-esurf',
 'gb-protein-1-4-eel',
 'gb-protein-eelect',
 'gb-protein-egb',
 'gb-protein-esurf',
 'gb-ligand-1-4-eel',
 'gb-ligand-eelec',
 'gb-ligand-egb',
 'gb-ligand-esurf',
 'pb-complex-vdwaals',
 'pb-protein-vdwaals',
 'pb-ligand-vdwaals']

In [8]:
df_all = pd.DataFrame({
    'complex-name': [],
    'gb-complex-1-4-eel': [],
    'gb-complex-eelec': [],
    'gb-complex-egb': [],
    'gb-complex-esurf': [],
    'gb-protein-1-4-eel': [],
    'gb-protein-eelect': [],
    'gb-protein-egb': [],
    'gb-protein-esurf': [],
    'gb-ligand-1-4-eel': [],
    'gb-ligand-eelec': [],
    'gb-ligand-egb': [],
    'gb-ligand-esurf': [],
    'pb-complex-vdwaals': [],
    'pb-protein-vdwaals': [],
    'pb-ligand-vdwaals': [],
    'ddg':[]
})

In [9]:
print(len(df_all.columns))



17


In [10]:
df_all

,complex-name,gb-complex-1-4-eel,gb-complex-eelec,gb-complex-egb,gb-complex-esurf,gb-protein-1-4-eel,gb-protein-eelect,gb-protein-egb,gb-protein-esurf,gb-ligand-1-4-eel,gb-ligand-eelec,gb-ligand-egb,gb-ligand-esurf,pb-complex-vdwaals,pb-protein-vdwaals,pb-ligand-vdwaals,ddg


In [11]:
df_values=df_info
ddg_arr=df_values['ddg'].to_numpy()


rows_list = []
for i in range(len(ddg_arr)):
    
    # Replace the column names in the row dictionary as requested
    row = {
        'complex-name': df_values.iloc[i]['complex-name'],
        'gb-complex-1-4-eel': df_values.iloc[i]['gb-complex-1-4-eel'],
        'gb-complex-eelec': df_values.iloc[i]['gb-complex-eelec'],
        'gb-complex-egb': df_values.iloc[i]['gb-complex-egb'],
        'gb-complex-esurf': df_values.iloc[i]['gb-complex-esurf'],
        'gb-protein-1-4-eel': df_values.iloc[i]['gb-protein-1-4-eel'],
        'gb-protein-eelect': df_values.iloc[i]['gb-protein-eelect'],
        'gb-protein-egb': df_values.iloc[i]['gb-protein-egb'],
        'gb-protein-esurf': df_values.iloc[i]['gb-protein-esurf'],
        'gb-ligand-1-4-eel': df_values.iloc[i]['gb-ligand-1-4-eel'],
        'gb-ligand-eelec': df_values.iloc[i]['gb-ligand-eelec'],
        'gb-ligand-egb': df_values.iloc[i]['gb-ligand-egb'],
        'gb-ligand-esurf': df_values.iloc[i]['gb-ligand-esurf'],
        'pb-complex-vdwaals': df_values.iloc[i]['pb-complex-vdwaals'],
        'pb-protein-vdwaals': df_values.iloc[i]['pb-protein-vdwaals'],
        'pb-ligand-vdwaals': df_values.iloc[i]['pb-ligand-vdwaals'],
        'ddg': df_values.iloc[i]['ddg']
    }

# Assuming the row will be used to append to a DataFrame or similar collection.

    
    rows_list.append(row)
# Step to concatenate the new rows into df_cd_set1
df_all = pd.concat([df_all, pd.DataFrame(rows_list)], ignore_index=True)

In [12]:
df_all

,complex-name,gb-complex-1-4-eel,gb-complex-eelec,gb-complex-egb,gb-complex-esurf,gb-protein-1-4-eel,gb-protein-eelect,gb-protein-egb,gb-protein-esurf,gb-ligand-1-4-eel,gb-ligand-eelec,gb-ligand-egb,gb-ligand-esurf,pb-complex-vdwaals,pb-protein-vdwaals,pb-ligand-vdwaals,ddg
0,10gs,18478.8142,-31052.1742,-3659.4630,86.9041,18034.9833,-30493.1722,-3672.7126,87.9920,443.8309,-416.8647,-146.3683,3.7375,-4143.6338,-4077.8698,-10.1928,-8.841927
1,1a1e,7030.7012,-13515.0036,-3349.3791,56.5130,7189.5223,-13366.9577,-3378.1703,56.2103,-158.8210,-96.6715,-69.1305,4.4598,-1951.5817,-1901.0486,-11.4008,-8.289306
2,1a30,9349.4510,-14111.7670,-2211.3839,48.1538,8949.8465,-13675.6929,-2204.7807,49.2906,399.6044,-381.1129,-265.2212,2.9765,-1837.5626,-1777.0400,-9.3186,-5.940670
3,1a4k,40100.7078,-61088.2896,-8626.6708,200.3571,40319.8406,-60991.3946,-8719.3884,201.0223,-219.1329,74.3491,-101.7143,3.4485,-1141.4576,-1084.3398,-6.0182,-11.052408
4,1a4r,16726.9825,-26744.9283,-4516.3615,97.5691,18021.0326,-27237.1053,-4529.6519,98.1097,-1294.0501,583.2385,-93.9965,3.0538,-1787.6642,-1733.7572,-5.8718,-9.201130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,7std,21176.0749,-35706.5273,-6267.2953,98.6928,21209.5397,-35704.9166,-6285.3439,99.6200,-33.4648,0.1221,-12.6225,2.7350,-5222.2144,-5167.3253,-4.6906,-14.810227
2731,7upj,8347.2909,-13713.5975,-1887.4373,49.7881,8615.8940,-13766.9993,-1988.7703,51.4697,-268.6031,102.8524,-32.5649,3.5941,-1740.3614,-1672.4526,-7.2150,-11.729368
2732,8a3h,14514.5548,-17122.3046,-8760.1462,55.9367,14306.8313,-16401.8897,-9310.9686,56.5806,207.7235,-84.0375,-104.9932,2.7302,-3095.4735,-3055.0110,-6.2099,-5.609097
2733,8cpa,13847.1770,-22434.5575,-2366.9434,58.2471,13472.1409,-21904.4816,-2475.0882,59.0256,375.0361,-439.7892,-102.3776,3.7517,-2345.7849,-2292.8024,-8.5007,-12.641192


In [ ]:
# df_values

In [14]:
import os
import pickle
import pandas as pd
from Bio import PDB

# Initialize PDB parser
parser = PDB.PDBParser(QUIET=True)

# List to store structures of matching PDBs
structures = []

pdb_directory = '/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/'

complex_value_name = list(df_info['complex-name'])
sum =0
for name in range(len(complex_value_name)):
    # Check if the complex_name matches any PDB file in the directory
    pdb_file = f"{complex_value_name[name]}.pdb"  # Assuming the PDB files are named after the complex_name
    if pdb_file in os.listdir(pdb_directory):  # Check if file exists in directory
        pdb_path = os.path.join(pdb_directory, pdb_file)

    # Check if the PDB file exists in the directory
    if os.path.exists(pdb_path):
        # Parse the PDB file
        print(pdb_path)
        sum+=1
        structure = parser.get_structure(pdb_file, pdb_path)
        structures.append((pdb_file, structure))  # Store as tuple (filename, structure)

# Pickle the matched PDB structures
with open('pdb_structures.pkl', 'wb') as f:
    pickle.dump(structures, f)

print("PDB files pickled successfully.")
print(sum)

        

/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/10gs.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a1e.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a30.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a4k.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a4r.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a4w.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a69.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a94.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a99.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1a9m.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1aaq.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1add.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1adl.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1ado.pdb
/home/lthoma21/BFE-Loss-Function/Datasets/pdbbind_complex/1afk

In [15]:
print(len(complex_value_name))

2735


In [ ]:
# import os
# import pickle
# from rdkit import Chem
# import pandas as pd

# # Define the directory containing PDB files
# pdb_directory = 'Datasets/pdbbind_complex/'

# # Example DataFrame (assuming it has a 'complex-name' column)
# df = new

# # List to store converted MOL objects
# mol_structures = []
# sum =0
# # Iterate through the DataFrame to get complex names
# for complex_name in df['complex-name']:
#     pdb_file = complex_name + '.pdb'  # Assuming PDB files are named after 'complex-name'

#     # Check if the corresponding PDB file exists in the directory
#     if pdb_file in os.listdir(pdb_directory):
#         pdb_path = os.path.join(pdb_directory, pdb_file)
#         print(pdb_path)

#         # Convert the PDB file to MOL format using RDKit
#         mol_structure = Chem.MolFromPDBFile(pdb_path, sanitize=False)
#         sum+=1
#         if mol_structure:  # Check if conversion was successful
#             mol_structures.append((pdb_file, mol_structure))  # Store the mol object with filename
            

# # Pickle the MOL structures
# with open('PDB_Mols_300.pkl', 'wb') as f:
#     pickle.dump(mol_structures, f)

# print("Matching PDB files converted to MOL and pickled successfully.")
# print(sum)